# Qualité de l'air de la station Châtelet
 
# Partie 3 : Modèle Random Forest

------------------------------

#### Sommaire

* Librairies
* Données
* Préparer les données : méthode du sliding window
* Modèle de Random Forest
* Annexe : Quelques mots sur le modèle Random Forest


# Librairies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#--------------------------------------------------#
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
pd.options.display.max_rows = 50
plt.rcParams['figure.figsize'] = [10,7]

# Données

Rappel : 
* nous avons choisi un échantillon de travail 'data_co2_ext' (air_chatelet_nbk_1_NETTOYAGE_DES_DONNEES.ipynb)
* nous l'avons ensuite séparé en deux parties : 'dataset' et 'valildation' (air_chatelet_nbk_2_ANALYSE_EXPLORATOIRE_DES_DONNEES.ipynb)
* nous utilisons ici 'dataset' pour construire nos modèles de prévision. 

In [3]:
dataset = pd.read_csv('datasets/dataset.csv', header=None, index_col=0, parse_dates=True, squeeze=True)

In [4]:
dataset.head()

0
2020-06-01 00:00:00+00:00    427.0
2020-06-01 01:00:00+00:00    421.0
2020-06-01 02:00:00+00:00    421.0
2020-06-01 03:00:00+00:00    421.0
2020-06-01 04:00:00+00:00    421.0
Name: 1, dtype: float64

# Préparer les données : méthode du sliding window

Avant de construire le modèle, on transforme les données sous la forme d'un problème d'apprentissage supervisé avec X et y :
* sortie y : les valeurs à un instant t
* entrée x : les valeurs aux instants précédents t-1, t-2, ... (un nombre de pas à choisir)

Par exemple, on choisit la fenêtre t-2, t-1, t pour former X et y.  
(voir l'exemple de sortie à la fin de cette partie) 

## Fonction adhoc

On construit une fonction pour préparer les données. 

### Fonction shift()

* On utilisera la fonction `shift(period)` qui s'applique à un dataframe (et retourne un dataframe)
    * si period > 0, alors les valeurs sont décalées vers le bas.
    * si period < 0, alors les valeurs sont décalées vers le haut. <br/><br/>
    
* Exemple : 

In [5]:
print("données de départ : ", pd.DataFrame(dataset).head())
print()
print("données avec un shift(1) : ", pd.DataFrame(dataset).shift(1).head())
print()
print("données avec un shift(-1) : ", pd.DataFrame(dataset).shift(-1).head())

données de départ :                                 1
0                               
2020-06-01 00:00:00+00:00  427.0
2020-06-01 01:00:00+00:00  421.0
2020-06-01 02:00:00+00:00  421.0
2020-06-01 03:00:00+00:00  421.0
2020-06-01 04:00:00+00:00  421.0

données avec un shift(1) :                                 1
0                               
2020-06-01 00:00:00+00:00    NaN
2020-06-01 01:00:00+00:00  427.0
2020-06-01 02:00:00+00:00  421.0
2020-06-01 03:00:00+00:00  421.0
2020-06-01 04:00:00+00:00  421.0

données avec un shift(-1) :                                 1
0                               
2020-06-01 00:00:00+00:00  421.0
2020-06-01 01:00:00+00:00  421.0
2020-06-01 02:00:00+00:00  421.0
2020-06-01 03:00:00+00:00  421.0
2020-06-01 04:00:00+00:00  422.0


### Construction de la fonction adhoc

La fonction ci-dessous est assez générale (et pourra être réutilisée dans d'autres contextes) : elle peut 
* prendre en entrée une séries avec plusieurs variables 
* considérer, pour X, un nombre d'instants précédents quelconque (t-1, t-2, ...)
* considérer, pour y, un nombre d'instants futures quelconque (t, t+1, t+2, ...).
    
D'abord on récupère le nombre de variables de la série en entrée :
* s'il s'agit d'une liste, il n'y a qu'une seule variable
* si c'est un array, on récupère le nombre via la 2e valeur du .shape().

On convertit la série en dataframe, pour pouvoir utiliser .shift().

On collecte les différents dataframes décalés (pour les temps passés t-1, t-2, ... et pour les temps futures t, t+1, ...), 
dans une liste, qu'on concatène à la fin. 

On élimine les lignes contenant des valeurs manquantes (celles du début après le .shift()).

In [6]:
# fonction pour transformer une série temporelle sous forme supervisée

'''
arguments :
    * data : séries de données sous de liste ou d'array
    * n_in : nombre d'instants passées à considérer pour construire X
    * n_out : nombre d'instants futures à considérer pour construire y
    * dropnan : booléen, pour supprimer les lignes contenant des valeurs manquantes
'''

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    # récupérer le nombre de variables de la série
    n_vars = 1 if type(data) is list else data.shape[1]
    # convertir les données en dataframe
    df = pd.DataFrame(data)
    # liste pour collecter les différents dataframes décalés avec .shift()
    cols = list()
    # collecter les dataframes décalés pour les instants passés (t-n_in, ... ,t-1) 
    for i in range(n_in, 0, -1):
        cols.append(df.shift(-i))  # period < 0 : décalage vers le haut
    # collecter les dataframes décalés pour les instants futures (t,t+1, ..., t+n) 
    for i in range(0, n_out):
        cols.append(df.shift(-i))  # period >= 0 : décalage vers le bas
    # concaténer les différents dataframes
    agg = pd.concat(cols, axis=1)
    # éliminer les lignes contenant des NaN
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

### Application de la fonction aux données

NB : 'dataset' est ici une Series, 'datasete.values' est un 1D-array, on la reshape pour avoir un 2D-array.

In [7]:
# on choisit ici par exemple de considérer 3 pas de temps dans le passé (t-3, t-2, t-1) pour former X
data = series_to_supervised(dataset.values.reshape(-1,1), n_in = 3)
data

array([[421., 421., 421., 427.],
       [421., 421., 421., 421.],
       [422., 421., 421., 421.],
       ...,
       [493., 544., 605., 632.],
       [474., 493., 544., 605.],
       [450., 474., 493., 544.]])

Les 3 premières colonnes représentent X, la dernière y. 

# Modèle de Random Forest

Des précisions sur le modèle sont données dans l'annexe.

Nous allons entraîner un modèle de Random Forest par 'walk-forward validation' sur nos données 'dataset' (essentiellement de la même manière que pour le modèle ARIMA). 

Plus précisément :
* on sépare les données en 2 parties : 
    * la 1e moitié est réservée à l'entraînement ('train')
    * la 2e moitié ('test') est utilisée de manière itérative pour pour l'entraînement et les prédictions
* plus précisément :     
    * on entraîne le modèle 
    * on utilise le modèle pour prédire la valeur à l'heure suivante
    * on ajoute la 1e valeur de 'test' traîtée dans 'train'
    * on recommence ces étapes.
    
A la fin, on évalue l'erreur de prédiction avec la RMSE.

## Fonctions adhoc

In [8]:
# séparer les données en train set et test set
'''
arguments :
    * data : données sous forme supervisée, un array-2D
    * n_test : nombre d'observations à garder pour le test set
'''
def train_test_split(data, n_test):
    train, test = data[:-n_test, :], data[-n_test:, :]
    return train, test

In [9]:
# entraîner le modèle de random forest et prédire la prochaine valeur
'''
arguments :
    * train : train set obtenue par la fonction précédente; array-2D
    * testX : test set obtenue par la fonction précédente, sans la partie y; array-2D
'''
def random_forest_forecast(train, testX):
    # transformer la liste en array
    train = np.asarray(train)
    # séparer les colonnes du trainset en entrée X et sortie y
    trainX, trainy = train[:,:-1], train[:,-1]
    # entraîner le modèle
    model = RandomForestRegressor(n_estimators=200)
    model.fit(trainX, trainy)
    # prédire l'instant suivant
    yhat = model.predict([testX]) 
    return yhat[0] 

In [10]:
# walk-forward validation 
'''
arguments :
    * data : données sous forme supervisée, un array-2D
    * n_test : nombre d'observations à garder pour le test set
'''
def walk_forward_validation(data, n_test):
    # liste pour collecter les prédictions
    predictions = list()
    # séparer en train et test
    train, test = train_test_split(data, n_test)
    # initialiser une liste avec les éléments de train
    history = [x for x in train]
    # itérer sur chaque élément de test
    for i in range(len(test)):
        # séparer test en entrée X et sortie y 
        testX, testy = test[i, :-1], test[i, -1]
        # entraîner le modèle sur history et prédire la valeur suivante
        yhat = random_forest_forecast(history, testX)
        # ajouter la prédiction dans la liste
        predictions.append(yhat)
        # ajouter l'observation en cours dans la liste
        history.append(test[i])
        # afficher pour vérification
        print('expected=%.1f, predicted=%.1f' % (testy, yhat))
    # estimer l'erreur de prédiction
    error = mean_squared_error(test[:, -1], predictions)
    return error, test[:, -1], predictions

## Appliquer les fonctions sur les données préparées

In [75]:
# on garde la moitié des données pour le test set
testsize = int(len(data) * 0.5)
rmse, y, yhat = walk_forward_validation(data, testsize)

expected=439.0, predicted=428.8
expected=428.0, predicted=427.1
expected=427.0, predicted=437.2
expected=435.0, predicted=437.7
expected=443.0, predicted=457.6
expected=457.0, predicted=463.9
expected=461.0, predicted=456.9
expected=470.0, predicted=473.4
expected=491.0, predicted=524.0
expected=537.0, predicted=554.7
expected=560.0, predicted=545.1
expected=565.0, predicted=527.5
expected=547.0, predicted=536.1
expected=546.0, predicted=543.1
expected=563.0, predicted=547.5
expected=572.0, predicted=564.3
expected=576.0, predicted=561.5
expected=606.0, predicted=607.1
expected=635.0, predicted=628.2
expected=650.0, predicted=611.0
expected=643.0, predicted=623.1
expected=599.0, predicted=552.0
expected=544.0, predicted=501.1
expected=490.0, predicted=465.5
expected=481.0, predicted=484.7
expected=477.0, predicted=490.1
expected=475.0, predicted=476.6
expected=472.0, predicted=460.8
expected=466.0, predicted=460.2
expected=465.0, predicted=458.2
expected=466.0, predicted=476.3
expected

expected=570.0, predicted=578.9
expected=580.0, predicted=571.9
expected=555.0, predicted=569.5
expected=538.0, predicted=555.9
expected=510.0, predicted=476.4
expected=456.0, predicted=447.4
expected=434.0, predicted=436.5
expected=429.0, predicted=431.4
expected=422.0, predicted=422.5
expected=418.0, predicted=425.5
expected=417.0, predicted=414.5
expected=414.0, predicted=413.2
expected=414.0, predicted=414.9
expected=417.0, predicted=423.4
expected=424.0, predicted=421.5
expected=431.0, predicted=457.6
expected=463.0, predicted=476.7
expected=478.0, predicted=492.2
expected=488.0, predicted=477.8
expected=480.0, predicted=483.0
expected=490.0, predicted=527.3
expected=512.0, predicted=501.9
expected=511.0, predicted=519.9
expected=536.0, predicted=569.9
expected=566.0, predicted=544.2
expected=566.0, predicted=578.9
expected=564.0, predicted=575.8
expected=543.0, predicted=553.6
expected=515.0, predicted=497.6
expected=459.0, predicted=444.5
expected=432.0, predicted=452.1
expected

expected=562.0, predicted=554.6
expected=563.0, predicted=519.5
expected=530.0, predicted=526.2
expected=526.0, predicted=546.0
expected=571.0, predicted=565.0
expected=578.0, predicted=562.8
expected=580.0, predicted=602.3
expected=616.0, predicted=595.2
expected=635.0, predicted=621.7
expected=647.0, predicted=630.0
expected=621.0, predicted=599.8
expected=577.0, predicted=536.8
expected=503.0, predicted=476.3
expected=471.0, predicted=477.8
expected=473.0, predicted=461.7
expected=462.0, predicted=470.0
expected=475.0, predicted=506.7
expected=497.0, predicted=526.0
expected=476.0, predicted=460.8
expected=456.0, predicted=455.4
expected=457.0, predicted=451.6
expected=459.0, predicted=468.3
expected=465.0, predicted=467.0
expected=474.0, predicted=473.9
expected=472.0, predicted=484.8
expected=485.0, predicted=506.3
expected=510.0, predicted=522.4
expected=533.0, predicted=557.9
expected=568.0, predicted=531.5
expected=563.0, predicted=556.1
expected=576.0, predicted=591.3
expected

expected=426.0, predicted=453.7
expected=442.0, predicted=441.3
expected=433.0, predicted=434.0
expected=430.0, predicted=424.9
expected=432.0, predicted=423.8
expected=452.0, predicted=487.7
expected=500.0, predicted=535.3
expected=553.0, predicted=544.3
expected=553.0, predicted=508.7
expected=512.0, predicted=499.2
expected=501.0, predicted=513.4
expected=528.0, predicted=530.5
expected=537.0, predicted=531.5
expected=531.0, predicted=530.7
expected=541.0, predicted=549.2
expected=579.0, predicted=628.9
expected=624.0, predicted=631.0
expected=592.0, predicted=548.8
expected=504.0, predicted=510.6
expected=473.0, predicted=466.4
expected=454.0, predicted=472.5
expected=450.0, predicted=452.0
expected=436.0, predicted=432.8
expected=431.0, predicted=427.8
expected=430.0, predicted=433.8
expected=435.0, predicted=443.1
expected=438.0, predicted=436.1
expected=436.0, predicted=432.2
expected=441.0, predicted=439.8
expected=463.0, predicted=491.5
expected=510.0, predicted=542.2
expected

expected=572.0, predicted=527.6
expected=488.0, predicted=469.7
expected=452.0, predicted=434.5
expected=436.0, predicted=447.6
expected=436.0, predicted=442.1
expected=435.0, predicted=440.9
expected=433.0, predicted=442.5
expected=427.0, predicted=423.6
expected=423.0, predicted=426.9
expected=424.0, predicted=424.2
expected=426.0, predicted=433.1
expected=432.0, predicted=429.2
expected=438.0, predicted=450.0
expected=455.0, predicted=451.7
expected=467.0, predicted=479.7
expected=490.0, predicted=498.0
expected=504.0, predicted=517.6
expected=534.0, predicted=567.9
expected=567.0, predicted=541.1
expected=562.0, predicted=572.5
expected=604.0, predicted=588.5
expected=592.0, predicted=554.6
expected=596.0, predicted=651.7
expected=713.0, predicted=615.1
expected=575.0, predicted=511.9
expected=478.0, predicted=457.8
expected=446.0, predicted=442.2
expected=440.0, predicted=436.4
expected=434.0, predicted=439.6
expected=436.0, predicted=437.9
expected=433.0, predicted=435.4
expected

...

expected=501.0, predicted=516.7
expected=487.0, predicted=465.1
expected=447.0, predicted=424.3
expected=423.0, predicted=423.4
expected=423.0, predicted=421.8
expected=424.0, predicted=433.8
expected=423.0, predicted=426.1
expected=423.0, predicted=423.0
expected=424.0, predicted=428.1
expected=427.0, predicted=422.4
expected=431.0, predicted=448.1
expected=449.0, predicted=457.9
expected=452.0, predicted=451.3
expected=454.0, predicted=455.9
expected=458.0, predicted=455.9
expected=465.0, predicted=490.9
expected=495.0, predicted=500.3
expected=507.0, predicted=500.8
expected=530.0, predicted=533.2
expected=537.0, predicted=536.4
expected=547.0, predicted=552.9
expected=557.0, predicted=551.9
expected=538.0, predicted=530.6
expected=503.0, predicted=560.2
expected=494.0, predicted=509.1
expected=474.0, predicted=446.8
expected=435.0, predicted=436.7
expected=434.0, predicted=437.6
expected=426.0, predicted=428.4
expected=421.0, predicted=430.5
expected=420.0, predicted=414.4
expected

expected=556.0, predicted=544.6
expected=557.0, predicted=533.2
expected=564.0, predicted=626.4
expected=640.0, predicted=607.2
expected=632.0, predicted=635.6
expected=650.0, predicted=629.0
expected=642.0, predicted=486.0
expected=529.0, predicted=560.6
expected=545.0, predicted=502.7
expected=480.0, predicted=489.1
expected=464.0, predicted=467.3
expected=455.0, predicted=445.5
expected=445.0, predicted=447.5
expected=446.0, predicted=452.5
expected=448.0, predicted=444.3
expected=447.0, predicted=446.4
expected=450.0, predicted=452.7
expected=469.0, predicted=457.1
expected=497.0, predicted=553.4
expected=555.0, predicted=540.5
expected=546.0, predicted=508.6
expected=520.0, predicted=557.3
expected=544.0, predicted=530.1
expected=546.0, predicted=543.1
expected=559.0, predicted=555.6
expected=566.0, predicted=556.9
expected=595.0, predicted=600.0
expected=624.0, predicted=572.5
expected=609.0, predicted=631.9
expected=624.0, predicted=553.4
expected=554.0, predicted=575.6
expected

expected=476.0, predicted=518.0
expected=526.0, predicted=531.8
expected=565.0, predicted=599.3
expected=577.0, predicted=519.8
expected=514.0, predicted=496.0
expected=503.0, predicted=517.1
expected=531.0, predicted=521.1
expected=533.0, predicted=542.0
expected=558.0, predicted=566.7
expected=585.0, predicted=564.6
expected=606.0, predicted=591.2
expected=628.0, predicted=631.8
expected=658.0, predicted=603.6
expected=597.0, predicted=558.5
expected=542.0, predicted=531.1
expected=496.0, predicted=480.0
expected=457.0, predicted=444.2
expected=431.0, predicted=419.3
expected=422.0, predicted=437.7
expected=433.0, predicted=440.2
expected=441.0, predicted=440.4
expected=438.0, predicted=438.0
expected=444.0, predicted=445.1
expected=455.0, predicted=444.4
expected=478.0, predicted=518.1
expected=531.0, predicted=558.4
expected=590.0, predicted=594.7
expected=586.0, predicted=526.6
expected=530.0, predicted=518.0
expected=531.0, predicted=533.1
expected=539.0, predicted=528.9
expected

expected=493.0, predicted=504.2
expected=470.0, predicted=425.1
expected=426.0, predicted=435.1
expected=434.0, predicted=439.0
expected=438.0, predicted=439.9
expected=439.0, predicted=439.6
expected=450.0, predicted=464.9
expected=478.0, predicted=530.1
expected=517.0, predicted=508.5
expected=529.0, predicted=538.5
expected=549.0, predicted=536.5
expected=500.0, predicted=451.5
expected=488.0, predicted=527.0
expected=539.0, predicted=518.0
expected=529.0, predicted=535.5
expected=545.0, predicted=552.7
expected=566.0, predicted=589.1
expected=607.0, predicted=597.7
expected=627.0, predicted=626.4
expected=638.0, predicted=525.2
expected=571.0, predicted=575.9
expected=565.0, predicted=503.1
expected=517.0, predicted=544.5
expected=519.0, predicted=524.4
expected=499.0, predicted=524.4
expected=491.0, predicted=449.7
expected=446.0, predicted=431.3
expected=433.0, predicted=436.0
expected=432.0, predicted=427.0
expected=431.0, predicted=435.6
expected=441.0, predicted=447.3
expected

expected=595.0, predicted=618.7
expected=639.0, predicted=603.0
expected=604.0, predicted=567.1
expected=557.0, predicted=573.3
expected=526.0, predicted=534.8
expected=485.0, predicted=473.4
expected=456.0, predicted=455.7
expected=441.0, predicted=452.0
expected=439.0, predicted=440.7
expected=432.0, predicted=440.6
expected=427.0, predicted=423.0
expected=420.0, predicted=420.8
expected=425.0, predicted=421.9
expected=428.0, predicted=428.7
expected=451.0, predicted=464.6
expected=491.0, predicted=554.9
expected=541.0, predicted=561.9
expected=547.0, predicted=488.8
expected=492.0, predicted=493.9
expected=497.0, predicted=522.6
expected=523.0, predicted=535.4
expected=545.0, predicted=535.8
expected=548.0, predicted=573.5
expected=601.0, predicted=572.5
expected=597.0, predicted=635.5
expected=703.0, predicted=675.4
expected=821.0, predicted=754.8
expected=850.0, predicted=617.7
expected=675.0, predicted=580.6
expected=522.0, predicted=507.7
expected=472.0, predicted=468.4
expected

In [76]:
rmse

621.2264990120676

⌛ Un temps d'entraînement assez long (plusieurs heures) pour un résultat pas terrible (avec ces paramètres) ...

# Annexe : Quelques mots sur le modèle Random Forest

|Random Forest||
|:---|:---|
|Idées| * Construire des arbres sur des échantillons différents et des variables différentes, pour qu'ils soient aussi différents des uns des autres que possible, afin que les erreurs puissent se compenser les uns des autres. <br/> * L'estimation finale est la moyenne des estimations de chaque arbre dans le cas d'un régression, ou le vote majoritaire dans le cas d'une classification. <br/> * On construit les différents échantillons à partir des données initiales par bootstrapping (tirage avec remise). <br/> * A chaque noeud, on choisit une partie des variables aléatoirement.|
|Quelques hyperparamètres|* n_estimators = nombre d'arbres (défaut : 100) <br/> * max_depth = profondeur maximale d'un arbre <br/> * min_samples_split : minimum d'observations à un noeud avant le split <br/> * min_samples_leaf = nombre minimum d'observations qu'une feuille doit avoir <br/> * max_features = nombre maximum de features à considérer lors de la recherche du meilleur split | 
|Avantages|* plutôt performant <br/> * dépend peu des hyperparamètres|
|Inconvénients| plus difficile à interpréter|

# Sources

* [Random Forest for Time Series Forecasting (machine learning mastery)](https://machinelearningmastery.com/random-forest-for-time-series-forecasting/)
* Introduction au Machine Learning - Chloé-Agathe Azencott
* Hands-On Machine Learning with Scikit-Learn, Kears and TensorFlow	- Aurélien Géron